In [16]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [19]:
preprocessor_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/2"

In [23]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(preprocessor_url)
encoder_inputs = preprocessor(text_input)

In [24]:
encoder = hub.KerasLayer(encoder_url, trainable=True)
outputs = encoder(encoder_inputs)

pooled_output = outputs["pooled_output"]
sequence_output = outputs["sequence_output"]

In [25]:
embedding_model = tf.keras.Model(text_input, pooled_output)
sentences = tf.constant(["Hello, BERT!", "I love machine learning"])
print(embedding_model(sentences))

tf.Tensor(
[[-9.87400413e-01  9.76773262e-01 -5.56680858e-02 -9.73757029e-01
  -9.91679013e-01 -7.14662850e-01  1.14679843e-01  1.44067913e-01
   9.20252562e-01 -7.31160820e-01 -9.76939499e-01  7.74911106e-01
   8.89673233e-01  9.89836156e-01  7.62470365e-02  9.00211453e-01
  -9.98005271e-01 -4.78055984e-01 -9.45764422e-01  9.41834807e-01
  -8.30212712e-01 -9.57827032e-01 -6.88169718e-01  9.09221947e-01
  -7.09947944e-01  8.52303028e-01 -8.87162328e-01 -9.75476027e-01
  -2.14520961e-01 -7.72614598e-01  9.81866598e-01  9.35896814e-01
   9.98023689e-01  8.00984859e-01  5.25259733e-01 -5.33965928e-03
  -9.46562707e-01 -4.30864304e-01  1.04046516e-01  5.03032804e-01
  -9.67220724e-01 -8.87948096e-01  1.86517298e-01  9.99273837e-01
  -5.40176272e-01 -1.62420690e-01 -8.57272267e-01 -9.70613718e-01
   9.55162704e-01  9.96276855e-01 -5.98798633e-01 -9.94817078e-01
   6.50841296e-01  1.37449712e-01  3.96825597e-02 -7.97589242e-01
   2.23712042e-01  1.97649240e-01 -9.22255635e-01  9.76661265e-01

In [26]:
sentences = tf.constant(["I love machine learning", "Machine learning is great"])
embeddings = embedding_model(sentences)
embeddings = tf.math.l2_normalize(embeddings, axis=1)
similarity = tf.matmul(embeddings, embeddings, transpose_b=True)
print(similarity.numpy())

[[1.        0.9272001]
 [0.9272001 1.0000001]]
